In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import seaborn as sns
import matplotlib as plt
import re
import numpy as np

In [ ]:
df1=pd.read_csv("../data/NSSCrisisv.1.csv")
df1.head()

In [ ]:
header_row = df1.iloc[1]
df1 = pd.DataFrame(df1.values[2:], columns=header_row)
df1.head()

In [ ]:
df2=pd.read_csv("../data/NSSCrisisv.2.csv")
df2.head(5)

In [ ]:
header_row = df2.iloc[1]
df2 = pd.DataFrame(df2.values[2:], columns=header_row)
df2.head()

In [ ]:
crisis_call=pd.ExcelFile("../data/NSS_Crisis_call.xlsx")
volume2020 = pd.read_excel(crisis_call, "2020")
volume2021 = pd.read_excel(crisis_call, "2021")
volume2022 = pd.read_excel(crisis_call, "2022")
volume2020.head()

In [ ]:
df1.info()

## Who works at the call center?

In [ ]:
workers=df1.PhoneWorkerName.unique()
workers

## How many calls did each worker take?

In [ ]:
work_call_vol=df1.PhoneWorkerName.value_counts()
work_call_vol

## Lets pull fix the dates and time

In [ ]:
df1.CallDateAndTimeStart	=df1.CallDateAndTimeStart	.str.split(" ")
df1.head(3)

## Close enough for horseshoes and hand grenades, so let's roll with it.
#### In the mean time, lets look at the average time each worker spent on a call. Does that yeild better results?

In [ ]:
df1.CallLength=df1.CallLength.astype('float')
df1.groupby('PhoneWorkerName').CallLength.mean()

In [ ]:
samecolumnsdf=pd.DataFrame(df1.columns.intersection(df2.columns))
# something Maggie shared. Maybe it's the merge we need right now
samecolumnsdf
#Nope, don't like that.

In [ ]:
df1.WasPrankCall.unique()
#what good is a column that is entirely null?

In [ ]:
pct_null = df1.isnull().sum() / len(df1)
missing_features = pct_null[pct_null > 0.98].index
df1.drop(missing_features, axis=1, inplace=True)
#Rob Schulteis found a way to drop columns that were 98%+ null

In [ ]:
df1.shape[1]
#87 columns is much more managable than 300 something

In [ ]:
df1.rename(columns={'CRISIS Demographics - Ethnicity': 'Ethnicity'}, inplace=True)
df2.rename(columns={'CRISIS Demographics - Race/Ethnicity': 'Ethnicity'}, inplace=True)
df1.Ethnicity.unique()

In [ ]:
df1.Ethnicity.value_counts()

In [ ]:
df1['CallDateAndTimeStart'].head()

#### I do not like these brackets. It makes this impossible to split. Let's ditch them.

In [ ]:
df1['CallDateAndTimeStart'] = df1['CallDateAndTimeStart'].str.get(0)
df1.head()

In [ ]:
roe=df1['CRISIS Call Information - Call Type'].value_counts()
roe=pd.DataFrame(roe)
roe

In [ ]:
roe.reset_index(inplace=True)
roe=roe.rename(columns={'index':'Call_Type', 'CRISIS Call Information - Call Type':'num'})
roe

In [ ]:
roe.info()

In [ ]:
df1['CRISIS Demographics - Education Level'].value_counts()

In [ ]:
df1['CRISIS Issues - Mental Health'].value_counts()

### Okay, enough of the value counts. Time to move on to somehting else.

In [ ]:
df1.Feedback.unique()

In [ ]:
volume2020

### 107  Spanish calls in 2020

In [ ]:
volume2021

### 248 crisis in Spanish in 2021

In [ ]:
volume2022

In [ ]:
### Spanish 65 Jan-Apr
65*4

###### 2020=107
2021=248
2022=260

In [ ]:
v20=pd.DataFrame(volume2020)
v21=pd.DataFrame(volume2021)

In [ ]:
#Pandemic_years=pd.concat(v20, v21, keys="Report Info")
#Pandemic_years

#### Analyze trends in call content by looking for which issues are most frequent and what is the average count of issues per call.

In [ ]:
labels = list()
for col in df2.columns:
    labels.append(col.replace('Subjective', 'Issues'))
df2.columns = labels
#for loop to change titles to match version 1 submitted by Rob S.

In [ ]:
df2.isnull().sum()

In [ ]:
df1.isnull().sum()

In [ ]:
df1_ci = df1[['CRISIS Issues - Abusive Behavior','CRISIS Issues - Emotional State','CRISIS Issues - Financial/Basic Needs','CRISIS Issues - Health/Physical','CRISIS Issues - Information or Services Needed','CRISIS Issues - Mental Health','CRISIS Issues - No Issue Call','CRISIS Issues - Other Description','CRISIS Issues - Relationships','CRISIS Issues - Substances','CRISIS Issues - Suicide']]
df1_ci.head()
#Jasmine Drumright narrows down to just Crisis issues.

In [ ]:
filter_col = [col for col in df2 if col.startswith('CRISIS Issues') or col.startswith('Call')]
df2_ci=df2[filter_col]
df2_ci.head()
#Rob's version of what Jasmine did for the second dataframe 

In [ ]:
Cri=pd.concat([df1_ci, df2_ci])
Cri

In [ ]:
Cri['CRISIS Issues - Emotional State'].value_counts()

In [ ]:
Cri['CRISIS Issues - Health/Physical'].value_counts()

In [ ]:
#Cri['CRISIS Issues - Financial/Basic Needs'].value_counts()
percent100 = Cri['CRISIS Issues - Financial/Basic Needs'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
percent100

In [ ]:
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('(temporary)','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace('(permanent)','')
Cri['CRISIS Issues - Financial/Basic Needs'] = Cri['CRISIS Issues - Financial/Basic Needs'].str.replace(r'\(\)','')
per_fin=Cri['CRISIS Issues - Financial/Basic Needs'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
per_fin

### Below is a loop of counts per item across columns by Rob Schulteis

In [ ]:
mental_health_df = Cri['CRISIS Issues - Mental Health'].notnull()

In [ ]:
mental_health_df = Cri[mental_health_df]

In [ ]:
mental_health_df.head(100)

In [ ]:
count = []

for x in mental_health_df['CRISIS Issues - Mental Health']:
    counter0 = x.count(";")+1
    count.append(counter0)

In [ ]:
mental_health_df['MH_Issues_Count'] = count

In [ ]:
mental_health_df.head()

### Finished with that Loop, now back to parsing out the data